In [1]:
#To ignore convergence warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
import cv2
import os
import pickle
import numpy as np
from time import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [3]:
def _data():
    #preprocess data
    #read images and convert to grayscale, then resize and flatten
    
    path='Data'
    label=['Negative', 'Positive']
    n=len(label)
    data=[[] for i in range(n)]
    target=[]
    for i in range(n):
        p=path+'//'+label[i]
        temp=os.listdir(p)
        count=0
        for j in range(len(temp)):
            text=temp[j].upper()
            if(not (text.endswith(".JPG") or text.endswith(".JPEG") or text.endswith(".PNG"))):
                temp=np.delete(temp, j-count)
                count+=1
        flag=0
        for j in temp:
            file=p+'//'+j
            #read image file
            img=cv2.imread(file)
            #convert to grayscale 
            img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #downsize
            img_=cv2.resize(img_gray, (50,50))
            #convert to array
            arr=np.array(img_)
            if(flag==0):
                temp=[arr]
                flag=1
            else:
                temp=np.vstack((temp,[arr]))
            data[i]=temp
            target.append(i)
    y=np.array(target)
    x=[]
    for i in range(n):
        for j in data[i]:
            arr=j
            #flatten each array 
            arr=arr.flatten()
            x.append(arr)
    x=np.array(x, float)
    return x,y

In [4]:
def _perform(y_test, y_predict):
    #calculates the performance metrics for the trained classifier's prediction
    
    tp=0
    tn=0
    fp=0
    fn=0
    s=np.shape(y_test)
    for i in range (s[0]):
        o1=y_predict[i]
        y1=y_test[i]
        if(o1==1 and y1==1):
            tp+=1
        elif(o1==0 and y1==0):
            tn+=1
        elif(o1==1 and y1==0):
            fp+=1
        else:
            fn+=1
    #Accuracy
    acc=(tp+tn)/(tp+tn+fp+fn)*100
    #Sensitivity or Recall
    sen=tp/(tp+fn)*100
    #Specificity
    spe=tn/(tn+fp)*100
    #Precision
    pre=tp/(tp+fp)*100
    #F1 Score
    f1=0.02*(sen*pre)/(sen+pre)
    return acc,sen,spe,pre,f1

In [5]:
#Linear Kernel

In [6]:
t0=time()
#set random seed
np.random.seed(1)
#import and preprocess data
x,y=_data()
#split data in train and test set; use stratification
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.4, stratify=y)
#instantiate PCA with 0.99 variance parameter; for feature extraction
pca=PCA(0.9)
#instantiate SVM classifier with linear kernel
clf=SVC(kernel='linear', max_iter=300)
#fit training data on PCA and transform train and test datasets
pca.fit(x_train)
x_train=pca.transform(x_train)
x_test=pca.transform(x_test)
#fit the classifier with training dataset
clf.fit(x_train, y_train)
#predict label for test data
y_predict=clf.predict(x_test)
#calculate performance for predicted vs expected labels
acc,sen,spe,pre,f1=_perform(y_test, y_predict)
print('Accuracy:\t',acc,'\nSpecificity:\t',spe,'\nSensitivity:\t',sen,'\nPrecision:\t',pre,'\nF1 Score:\t',f1)
print('Time:\t', time()-t0)

Accuracy:	 70.9090909090909 
Specificity:	 76.31578947368422 
Sensitivity:	 58.82352941176471 
Precision:	 52.63157894736842 
F1 Score:	 0.5555555555555555
Time:	 4.918268918991089


In [7]:
# save the PCA
with open('../Webapp/pca.pkl', 'wb') as pkl1:
    pickle.dump(pca, pkl1)
# save the classifier
with open('../Webapp/svc.pkl', 'wb') as pkl2:
    pickle.dump(clf, pkl2)